# Connect 4 Deep Search Dataset Generator

**Goal**: Generate 5,000+ deep MCTS games in 2-3 hours for training data.

## Key Features
- **Deeper MCTS** with heuristic rollouts (500-5,000 sims vs ULTRAFAST's 100-250)
- **Multiprocessing** to use all available CPU cores in parallel
- **Dual-mode**: works on Google Colab (high-RAM CPU) and local Jupyter
- **Time-capped** with periodic checkpointing -- never lose progress
- **6x7x2 encoding** (option b) compatible with existing training pipeline

## How to Use
1. Run all cells in order
2. Adjust `MAX_RUNTIME_MINUTES` and `TARGET_GAMES` in the Configuration cell
3. Dataset saves automatically as `.npz` files

## Cell 1: Auto-Detect Environment & Setup Directories

In [ ]:
import os
import sys

# ---------- Auto-detect Colab vs Local ----------
IS_COLAB = False
try:
    import google.colab
    IS_COLAB = True
except ImportError:
    pass

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_DIR = '/content/drive/MyDrive/Connect4_DeepSearch'
    print("Running on Google Colab")
else:
    # Local Jupyter -- save next to this notebook
    PROJECT_DIR = os.path.join(os.getcwd(), 'Connect4_DeepSearch_Output')
    print("Running on Local Jupyter")

DATASET_DIR = os.path.join(PROJECT_DIR, 'datasets')
CHECKPOINT_DIR = os.path.join(PROJECT_DIR, 'checkpoints')
LOG_DIR = os.path.join(PROJECT_DIR, 'logs')

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

print(f"Project directory : {PROJECT_DIR}")
print(f"Dataset directory : {DATASET_DIR}")
print(f"Checkpoint directory: {CHECKPOINT_DIR}")

## Cell 2: Imports & Resource Check

In [ ]:
import numpy as np
import random
import pickle
import json
import time
import multiprocessing as mp
from datetime import datetime
from collections import OrderedDict
import gc
import matplotlib
matplotlib.use('Agg')  # non-interactive backend (safe for Colab + local)
import matplotlib.pyplot as plt

# Optional: tqdm for progress bars
try:
    from tqdm.auto import tqdm
    HAS_TQDM = True
except ImportError:
    HAS_TQDM = False
    print("tqdm not found -- progress bars will be text-based")

# ---------- Resource info ----------
NUM_CPUS = os.cpu_count() or 2
print(f"Available CPUs: {NUM_CPUS}")

try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"RAM: {mem.available / (1024**3):.1f} GB available / {mem.total / (1024**3):.1f} GB total ({mem.percent}% used)")
except ImportError:
    print("psutil not installed -- skipping detailed memory info")

print(f"NumPy version: {np.__version__}")
print("Imports loaded successfully")

## Cell 3: Configuration

Adjust these parameters to control depth, speed, and runtime.

In [ ]:
# ==================== CONFIGURATION ====================

# --- Target ---
TARGET_GAMES = 6000          # Aim for 6k to guarantee >= 5k even if time-capped
MAX_RUNTIME_MINUTES = 150    # Hard time cap (2.5 hrs, leaves buffer within 3 hrs)

# --- MCTS Depth Bands ---
# (probability, min_sims, max_sims)
# Weighted so the average depth is ~1000 sims -- significantly deeper than ULTRAFAST's 150
DEPTH_BANDS = [
    (0.60,  500,  1000),   # Solid midrange depth
    (0.25, 1000,  2000),   # Strong depth
    (0.10, 2000,  3500),   # Very strong
    (0.05, 3500,  5000),   # Expert-level depth
]

# --- Opening Randomness (for diversity) ---
RANDOM_PROB = 0.15           # Probability of random move in opening phase
RANDOM_DEPTH = 10            # Only apply random moves in first N moves of a game
STRONG_EPSILON = 0.02        # Small chance of random MCTS move for variety

# --- Multiprocessing ---
# Use min(cpu_count, 4) to avoid overwhelming the system
NUM_WORKERS = min(NUM_CPUS, 4)
BATCH_PER_WORKER = 50        # Each worker generates this many games per batch

# --- Checkpointing ---
CHECKPOINT_INTERVAL_MINUTES = 20   # Save progress every N minutes

# --- Reproducibility ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# ==================== PRINT SUMMARY ====================
avg_sims = sum(p * (lo + hi) / 2 for p, lo, hi in DEPTH_BANDS)
print("=" * 55)
print("  DEEP SEARCH DATASET GENERATOR - Configuration")
print("=" * 55)
print(f"  Target games       : {TARGET_GAMES:,}")
print(f"  Max runtime (min)  : {MAX_RUNTIME_MINUTES}")
print(f"  Avg MCTS sims/move : ~{avg_sims:.0f}")
print(f"  Depth bands        : {DEPTH_BANDS}")
print(f"  Random prob        : {RANDOM_PROB}")
print(f"  Random depth       : {RANDOM_DEPTH}")
print(f"  Strong epsilon     : {STRONG_EPSILON}")
print(f"  Workers            : {NUM_WORKERS}")
print(f"  Batch per worker   : {BATCH_PER_WORKER}")
print(f"  Checkpoint interval: {CHECKPOINT_INTERVAL_MINUTES} min")
print(f"  Seed               : {SEED}")
print("=" * 55)

## Cell 4: Fast Connect4 Engine

Optimized engine with `heights` array to avoid recomputing column fill levels on every move.

In [ ]:
class Connect4:
    """Fast Connect 4 engine with height-tracking for O(1) move placement."""

    __slots__ = ['board', 'heights', 'current_player', 'winner', 'move_count']

    def __init__(self):
        self.board = np.zeros((6, 7), dtype=np.int8)
        self.heights = np.zeros(7, dtype=np.int8)   # tracks how many pieces in each column
        self.current_player = 1
        self.winner = None
        self.move_count = 0

    def copy(self):
        g = Connect4()
        g.board = self.board.copy()
        g.heights = self.heights.copy()
        g.current_player = self.current_player
        g.winner = self.winner
        g.move_count = self.move_count
        return g

    def legal_moves(self):
        """Return list of columns that are not full."""
        return [c for c in range(7) if self.heights[c] < 6]

    def make_move(self, col):
        """Place piece in column. Returns False if illegal."""
        if col < 0 or col > 6 or self.heights[col] >= 6:
            return False
        row = int(self.heights[col])
        self.board[row, col] = self.current_player
        self.heights[col] += 1
        self.move_count += 1

        if self._check_win(row, col):
            self.winner = self.current_player
        elif self.move_count >= 42:
            self.winner = 0  # draw

        self.current_player *= -1
        return True

    def _check_win(self, row, col):
        """Check if last move at (row, col) created a 4-in-a-row."""
        player = self.board[row, col]

        # Vertical (only need to check downward since piece sits on top)
        if row >= 3:
            if (self.board[row, col] + self.board[row-1, col] +
                self.board[row-2, col] + self.board[row-3, col]) == 4 * player:
                return True

        # Horizontal
        for start in range(max(0, col - 3), min(4, col + 1)):
            if np.sum(self.board[row, start:start+4]) == 4 * player:
                return True

        # Diagonals (extend in both directions)
        for dr, dc in [(1, 1), (1, -1)]:
            count = 1
            for sign in [1, -1]:
                r, c = row + sign * dr, col + sign * dc
                while 0 <= r < 6 and 0 <= c < 7 and self.board[r, c] == player:
                    count += 1
                    r += sign * dr
                    c += sign * dc
            if count >= 4:
                return True

        return False

    def is_terminal(self):
        return self.winner is not None

    def get_result(self, player):
        """Return +1 if player won, -1 if lost, 0 if draw or not finished."""
        if self.winner is None:
            return 0.0
        if self.winner == 0:
            return 0.0
        return 1.0 if self.winner == player else -1.0

    def encode(self):
        """6x7x2 encoding (option b from project description)."""
        enc = np.zeros((6, 7, 2), dtype=np.float32)
        enc[:, :, 0] = (self.board == 1).astype(np.float32)
        enc[:, :, 1] = (self.board == -1).astype(np.float32)
        return enc

    def board_hash(self):
        """Fast hash for transposition lookups."""
        return hash(self.board.tobytes())

print("Connect4 engine loaded")

## Cell 5: MCTS with Heuristic Rollouts

the agent checks for immediate wins and blocks before falling back to random moves. This gives much higher quality evaluations per simulation.
I think it reduces by effectively making 500 heuristic sims comparable to 1,500+ random sims.

In [ ]:
class DeepMCTS:
    """MCTS with heuristic rollouts, win/block shortcuts, and UCB1 selection."""

    def __init__(self, default_sims=800):
        self.default_sims = default_sims

    # ---------- Public API ----------

    def get_move(self, game, sims=None):
        """Return the best column to play."""
        sims = sims or self.default_sims
        player = game.current_player

        # Instant win check
        win_col = self._find_winning_move(game, player)
        if win_col is not None:
            return win_col

        # Instant block check
        block_col = self._find_winning_move(game, -player)
        if block_col is not None:
            return block_col

        # Full MCTS search
        return self._search(game, sims)

    # ---------- MCTS core ----------

    def _search(self, game, sims):
        root_player = game.current_player
        # stats: board_hash -> [visits, value_sum]
        stats = {}

        for _ in range(sims):
            node = game.copy()
            path = []

            # --- Selection + Expansion ---
            while not node.is_terminal():
                state = node.board_hash()
                path.append(state)

                if state not in stats:
                    stats[state] = [0, 0.0]

                moves = node.legal_moves()
                if not moves:
                    break

                # UCB1 selection
                best_move = None
                best_ucb = -1e18
                parent_visits = stats[state][0]

                for col in moves:
                    test = node.copy()
                    test.make_move(col)
                    child_hash = test.board_hash()

                    if child_hash not in stats:
                        stats[child_hash] = [0, 0.0]

                    visits, value = stats[child_hash]

                    if visits == 0:
                        ucb = 1e18  # explore unvisited first
                    else:
                        exploit = value / visits
                        explore = 1.414 * np.sqrt(np.log(parent_visits + 1) / visits)
                        ucb = exploit + explore

                    if ucb > best_ucb:
                        best_ucb = ucb
                        best_move = col

                node.make_move(best_move)

                # If this is a new node, expand and rollout
                child_state = node.board_hash()
                if child_state not in stats or stats[child_state][0] == 0:
                    if child_state not in stats:
                        stats[child_state] = [0, 0.0]
                    path.append(child_state)
                    break

            # --- Heuristic Rollout ---
            result = self._heuristic_rollout(node, root_player)

            # --- Backpropagation ---
            for state in path:
                if state in stats:
                    stats[state][0] += 1
                    stats[state][1] += result

        # --- Choose best move by visit count ---
        best_move = None
        best_visits = -1

        for col in game.legal_moves():
            test = game.copy()
            test.make_move(col)
            st = test.board_hash()

            visits = stats[st][0] if st in stats else 0
            if visits > best_visits:
                best_visits = visits
                best_move = col

        return best_move if best_move is not None else random.choice(game.legal_moves())

    # ---------- Heuristic rollout ----------

    def _heuristic_rollout(self, game, root_player, max_depth=30):
        """
        Rollout with heuristic move ordering:
        1. Play winning move if available
        2. Block opponent's winning move
        3. Prefer center columns
        4. Random fallback
        """
        node = game.copy()
        depth = 0

        while not node.is_terminal() and depth < max_depth:
            moves = node.legal_moves()
            if not moves:
                break

            player = node.current_player

            # 1) Check for immediate win
            win_col = self._find_winning_move(node, player)
            if win_col is not None:
                node.make_move(win_col)
                depth += 1
                continue

            # 2) Check for immediate block
            block_col = self._find_winning_move(node, -player)
            if block_col is not None:
                node.make_move(block_col)
                depth += 1
                continue

            # 3) Center-biased random (center columns are strategically better)
            #    Column order: 3, 2, 4, 1, 5, 0, 6
            center_order = [3, 2, 4, 1, 5, 0, 6]
            available_center = [c for c in center_order if c in moves]

            if random.random() < 0.6 and available_center:
                # 60% of the time, bias toward center
                col = available_center[0]
            else:
                col = random.choice(moves)

            node.make_move(col)
            depth += 1

        return node.get_result(root_player)

    # ---------- Utility ----------

    def _find_winning_move(self, game, player):
        """Check if `player` can win in one move. Returns column or None."""
        for col in game.legal_moves():
            test = game.copy()
            # Temporarily set the player so make_move uses the right piece
            saved_player = test.current_player
            test.current_player = player
            test.make_move(col)
            if test.winner == player:
                return col
            # Restore (not strictly needed since we discard `test`)
        return None


# ---------- Depth sampler ----------

def sample_depth():
    """Sample MCTS simulation count from the configured depth bands."""
    r = random.random()
    acc = 0.0
    for pct, lo, hi in DEPTH_BANDS:
        acc += pct
        if r <= acc:
            return random.randint(lo, hi)
    # Fallback to last band
    return random.randint(DEPTH_BANDS[-1][1], DEPTH_BANDS[-1][2])

print("DeepMCTS engine loaded")
print(f"Avg sims per move: ~{sum(p * (lo+hi)/2 for p, lo, hi in DEPTH_BANDS):.0f}")

## Cell 6: Single Game Generator

Plays one full game of Connect 4 using Deep MCTS. Random opening moves create diversity
but are NOT recorded in the dataset (per project spec -- only MCTS-recommended moves are stored).

In [ ]:
def generate_one_game(seed_offset=0):
    """
    Play a single game and return (boards_list, moves_list, metadata_dict).

    - Random opening moves are played but NOT recorded.
    - MCTS-recommended moves ARE recorded with the board encoded from
      player 1's perspective (flipping for player -1).
    - Each worker gets a unique seed_offset for reproducibility.
    """
    # Per-call seed so each multiprocessing worker is independent
    local_rng = random.Random(SEED + seed_offset + int(time.time() * 1000) % 100000)
    np_rng = np.random.RandomState(SEED + seed_offset + int(time.time() * 1000) % 100000)

    mcts_player = DeepMCTS()
    game = Connect4()

    game_boards = []
    game_moves = []
    depths_used = []
    move_num = 0

    while not game.is_terminal():
        current_player = game.current_player

        # Random opening phase -- diverse starts, not recorded
        use_random = (move_num < RANDOM_DEPTH and local_rng.random() < RANDOM_PROB)

        if use_random:
            col = local_rng.choice(game.legal_moves())
        else:
            # Sample depth from bands
            depth = sample_depth()
            depths_used.append(depth)

            # Small epsilon for variety
            if local_rng.random() < STRONG_EPSILON:
                col = local_rng.choice(game.legal_moves())
            else:
                col = mcts_player.get_move(game, sims=depth)

            # Encode board from player 1 perspective
            if current_player == 1:
                board_enc = game.encode()
            else:
                flipped = game.copy()
                flipped.board = -flipped.board
                board_enc = flipped.encode()

            game_boards.append(board_enc)
            game_moves.append(col)

        game.make_move(col)
        move_num += 1

        if move_num > 42:
            break

    # Game metadata
    avg_depth = sum(depths_used) / len(depths_used) if depths_used else 0
    result = 'p1_win' if game.winner == 1 else ('p2_win' if game.winner == -1 else 'draw')

    meta = {
        'avg_depth': avg_depth,
        'max_depth': max(depths_used) if depths_used else 0,
        'total_moves': move_num,
        'recorded_positions': len(game_boards),
        'result': result,
    }

    return game_boards, game_moves, meta


def generate_batch(args):
    """
    Worker function for multiprocessing.
    Generates `num_games` games and returns aggregated results.
    """
    batch_id, num_games, start_time_global, max_runtime_sec = args

    batch_boards = []
    batch_moves = []
    batch_metas = []
    games_done = 0

    for i in range(num_games):
        # Check time cap
        if time.time() - start_time_global >= max_runtime_sec:
            break

        boards, moves, meta = generate_one_game(seed_offset=batch_id * 10000 + i)
        batch_boards.extend(boards)
        batch_moves.extend(moves)
        batch_metas.append(meta)
        games_done += 1

    return batch_boards, batch_moves, batch_metas, games_done

print("Game generator loaded")

## Cell 7: Parallel Dataset Generator

Uses multiprocessing to generate games on all available CPU cores simultaneously.
Includes periodic checkpointing (every 20 min) and a hard time cap.

In [ ]:
def _try_multiprocessing():
    """
    Test whether multiprocessing works in this environment.
    Jupyter notebooks + 'spawn' context often can't pickle notebook-defined functions.
    Linux (Colab) with 'fork' usually works. macOS defaults to 'spawn' which doesn't.
    Returns True if MP is usable, False otherwise.
    """
    if NUM_WORKERS <= 1:
        return False
    try:
        ctx = mp.get_context('fork')
        with ctx.Pool(1) as pool:
            result = pool.map(generate_batch, [(0, 1, time.time(), 600)])
        return True
    except Exception:
        pass
    # Try spawn as fallback (usually fails in notebooks but worth trying)
    try:
        ctx = mp.get_context('spawn')
        with ctx.Pool(1) as pool:
            result = pool.map(generate_batch, [(0, 1, time.time(), 600)])
        return True
    except Exception:
        return False


def _run_parallel_batch(batch_id, games_per_worker, start_time, max_runtime_sec, n_workers):
    """Run a batch of games in parallel using multiprocessing fork context."""
    worker_args = [
        (batch_id * n_workers + w, games_per_worker, start_time, max_runtime_sec)
        for w in range(n_workers)
    ]
    ctx = mp.get_context('fork')
    with ctx.Pool(processes=n_workers) as pool:
        results = pool.map(generate_batch, worker_args)
    return results


def generate_deep_dataset():
    """
    Main generation loop.
    - Attempts multiprocessing (fork context); falls back to sequential if unavailable
    - Collects results and checkpoints periodically
    - Stops when TARGET_GAMES reached or time cap hit
    """
    all_boards = []
    all_moves = []
    all_metas = []
    total_games = 0

    start_time = time.time()
    max_runtime_sec = MAX_RUNTIME_MINUTES * 60
    last_checkpoint_time = start_time
    checkpoint_num = 0

    log_lines = []

    def log(msg):
        ts = datetime.now().strftime('%H:%M:%S')
        line = f"[{ts}] {msg}"
        print(line)
        log_lines.append(line)

    log(f"Starting deep dataset generation")
    log(f"Target: {TARGET_GAMES:,} games | Time cap: {MAX_RUNTIME_MINUTES} min | Workers: {NUM_WORKERS}")
    log(f"Avg MCTS depth: ~{sum(p*(lo+hi)/2 for p,lo,hi in DEPTH_BANDS):.0f} sims/move")

    # Test multiprocessing availability
    use_mp = _try_multiprocessing()
    if use_mp:
        log(f"Multiprocessing available -- using {NUM_WORKERS} workers (fork context)")
    else:
        log("Multiprocessing not available in this environment -- using single-process mode")
        log("(This is normal for macOS Jupyter. On Colab Linux, fork context should work.)")

    batch_id = 0

    while total_games < TARGET_GAMES:
        elapsed_sec = time.time() - start_time
        if elapsed_sec >= max_runtime_sec:
            log("Time cap reached. Stopping generation.")
            break

        remaining = TARGET_GAMES - total_games
        remaining_time_sec = max_runtime_sec - elapsed_sec
        if remaining_time_sec <= 0:
            break

        if use_mp:
            games_per_worker = min(BATCH_PER_WORKER, max(1, remaining // NUM_WORKERS))
            try:
                results = _run_parallel_batch(
                    batch_id, games_per_worker, start_time, max_runtime_sec, NUM_WORKERS
                )
                for boards, moves, metas, games_done in results:
                    all_boards.extend(boards)
                    all_moves.extend(moves)
                    all_metas.extend(metas)
                    total_games += games_done
            except Exception as e:
                log(f"Multiprocessing error: {e}. Falling back to sequential.")
                use_mp = False
                continue
        else:
            # Sequential fallback
            games_this_batch = min(BATCH_PER_WORKER, remaining)
            boards, moves, metas, games_done = generate_batch(
                (batch_id, games_this_batch, start_time, max_runtime_sec)
            )
            all_boards.extend(boards)
            all_moves.extend(moves)
            all_metas.extend(metas)
            total_games += games_done

        batch_id += 1

        # Progress report
        elapsed_min = (time.time() - start_time) / 60.0
        rate = total_games / max(elapsed_min, 0.01)
        eta_min = (TARGET_GAMES - total_games) / max(rate, 0.01)
        log(f"Games: {total_games:,}/{TARGET_GAMES:,} | "
            f"Examples: {len(all_boards):,} | "
            f"Rate: {rate:.1f} games/min | "
            f"Elapsed: {elapsed_min:.1f} min | "
            f"ETA: {eta_min:.1f} min")

        # Periodic checkpoint
        if (time.time() - last_checkpoint_time) / 60.0 >= CHECKPOINT_INTERVAL_MINUTES:
            checkpoint_num += 1
            ckpt_path = os.path.join(CHECKPOINT_DIR, f'deep_ckpt_{checkpoint_num:03d}.pkl')
            with open(ckpt_path, 'wb') as f:
                pickle.dump({
                    'boards': all_boards,
                    'moves': all_moves,
                    'games': total_games,
                    'timestamp': datetime.now().isoformat(),
                }, f)
            log(f"Checkpoint {checkpoint_num} saved -> {ckpt_path}")
            last_checkpoint_time = time.time()
            gc.collect()

    # Final checkpoint
    checkpoint_num += 1
    ckpt_path = os.path.join(CHECKPOINT_DIR, f'deep_ckpt_{checkpoint_num:03d}.pkl')
    with open(ckpt_path, 'wb') as f:
        pickle.dump({
            'boards': all_boards,
            'moves': all_moves,
            'games': total_games,
            'timestamp': datetime.now().isoformat(),
        }, f)
    log(f"Final checkpoint {checkpoint_num} saved -> {ckpt_path}")

    # Compute summary stats
    elapsed_total = (time.time() - start_time) / 60.0
    p1_wins = sum(1 for m in all_metas if m['result'] == 'p1_win')
    p2_wins = sum(1 for m in all_metas if m['result'] == 'p2_win')
    draws = sum(1 for m in all_metas if m['result'] == 'draw')
    avg_depth = np.mean([m['avg_depth'] for m in all_metas]) if all_metas else 0
    avg_positions = np.mean([m['recorded_positions'] for m in all_metas]) if all_metas else 0

    log("=" * 55)
    log("GENERATION COMPLETE")
    log("=" * 55)
    log(f"Total games     : {total_games:,}")
    log(f"Total examples  : {len(all_boards):,}")
    log(f"Total time      : {elapsed_total:.1f} min ({elapsed_total/60:.2f} hrs)")
    log(f"Rate            : {total_games/max(elapsed_total,0.01):.1f} games/min")
    log(f"Avg MCTS depth  : {avg_depth:.0f} sims/move")
    log(f"Avg positions   : {avg_positions:.1f} per game")
    log(f"P1 wins: {p1_wins} ({100*p1_wins/max(total_games,1):.1f}%) | "
        f"P2 wins: {p2_wins} ({100*p2_wins/max(total_games,1):.1f}%) | "
        f"Draws: {draws} ({100*draws/max(total_games,1):.1f}%)")

    # Save log
    log_path = os.path.join(LOG_DIR, f'generation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    with open(log_path, 'w') as f:
        f.write('\n'.join(log_lines))
    print(f"Log saved -> {log_path}")

    return np.array(all_boards), np.array(all_moves), all_metas

print("Dataset generator ready")

## Cell 8: RUN GENERATION (Main Cell)

This is the main execution cell. It will run for up to `MAX_RUNTIME_MINUTES` and
generate as many deep games as possible. Progress is printed periodically.

In [ ]:
print("=" * 55)
print("  STARTING DEEP DATASET GENERATION")
print("=" * 55)

X_raw, y_raw, game_metas = generate_deep_dataset()

print(f"\nRaw dataset:")
print(f"  X shape: {X_raw.shape}")
print(f"  y shape: {y_raw.shape}")

## Cell 9: Data Augmentation (Horizontal Flip)

Doubles the dataset by mirroring every board and its corresponding move.

In [ ]:
def augment_dataset(boards, moves):
    """Horizontal flip augmentation -- doubles the dataset."""
    n = len(boards)
    aug_boards = np.zeros((n * 2, 6, 7, 2), dtype=np.float32)
    aug_moves = np.zeros(n * 2, dtype=np.int64)

    print(f"Augmenting {n:,} examples with horizontal flips...")

    for i in range(n):
        # Original
        aug_boards[2 * i] = boards[i]
        aug_moves[2 * i] = moves[i]

        # Horizontally flipped
        aug_boards[2 * i + 1] = np.flip(boards[i], axis=1)
        aug_moves[2 * i + 1] = 6 - moves[i]

    return aug_boards, aug_moves


X_aug, y_aug = augment_dataset(X_raw, y_raw)

print(f"\nAugmentation complete!")
print(f"  Original : {len(X_raw):,} examples")
print(f"  Augmented: {len(X_aug):,} examples")

## Cell 10: Quality Analysis & Visualization

In [ ]:
# ---------- Statistics ----------
print("=" * 55)
print("  DATASET QUALITY REPORT")
print("=" * 55)
print(f"Total examples (augmented): {len(X_aug):,}")
print(f"X shape: {X_aug.shape}")
print(f"y shape: {y_aug.shape}")

print(f"\nMove distribution:")
for col in range(7):
    count = np.sum(y_aug == col)
    pct = 100 * count / len(y_aug)
    bar = '#' * int(pct)
    print(f"  Column {col}: {count:>7,} ({pct:5.1f}%) {bar}")

# Game outcome stats
if game_metas:
    p1w = sum(1 for m in game_metas if m['result'] == 'p1_win')
    p2w = sum(1 for m in game_metas if m['result'] == 'p2_win')
    drw = sum(1 for m in game_metas if m['result'] == 'draw')
    total = len(game_metas)
    print(f"\nGame outcomes ({total:,} games):")
    print(f"  P1 wins: {p1w:,} ({100*p1w/total:.1f}%)")
    print(f"  P2 wins: {p2w:,} ({100*p2w/total:.1f}%)")
    print(f"  Draws  : {drw:,} ({100*drw/total:.1f}%)")

    depths = [m['avg_depth'] for m in game_metas]
    print(f"\nMCTS depth stats:")
    print(f"  Mean : {np.mean(depths):.0f} sims")
    print(f"  Min  : {np.min(depths):.0f} sims")
    print(f"  Max  : {np.max(depths):.0f} sims")
    print(f"  Median: {np.median(depths):.0f} sims")

# ---------- Visualizations ----------
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1) Move distribution
move_counts = np.bincount(y_aug.astype(int), minlength=7)
axes[0, 0].bar(range(7), move_counts, color='steelblue', edgecolor='black')
axes[0, 0].set_title('Move Distribution (Augmented)', fontweight='bold')
axes[0, 0].set_xlabel('Column')
axes[0, 0].set_ylabel('Count')

# 2) Game stage coverage (pieces on board)
piece_counts = [np.sum(board) for board in X_aug[:min(len(X_aug), 50000)]]
axes[0, 1].hist(piece_counts, bins=30, color='green', edgecolor='black', alpha=0.8)
axes[0, 1].set_title('Game Stage Coverage', fontweight='bold')
axes[0, 1].set_xlabel('Total Pieces on Board')
axes[0, 1].set_ylabel('Frequency')

# 3) MCTS depth distribution
if game_metas:
    axes[1, 0].hist(depths, bins=40, color='purple', edgecolor='black', alpha=0.8)
    axes[1, 0].set_title('MCTS Depth Distribution', fontweight='bold')
    axes[1, 0].set_xlabel('Avg Sims per Game')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].axvline(np.mean(depths), color='red', linestyle='--', label=f'Mean: {np.mean(depths):.0f}')
    axes[1, 0].legend()

# 4) Sample board visualization
idx = random.randint(0, len(X_aug) - 1)
sample_board = X_aug[idx]
board_2d = sample_board[:, :, 0] - sample_board[:, :, 1]
im = axes[1, 1].imshow(board_2d, cmap='RdBu', vmin=-1, vmax=1)
axes[1, 1].set_title(f'Sample Board (Move: Col {y_aug[idx]})', fontweight='bold')
axes[1, 1].set_xlabel('Column')
axes[1, 1].set_ylabel('Row')
plt.colorbar(im, ax=axes[1, 1], shrink=0.8)

plt.tight_layout()
analysis_path = os.path.join(PROJECT_DIR, 'deep_dataset_analysis.png')
plt.savefig(analysis_path, dpi=150, bbox_inches='tight')
plt.show()
print(f"\nAnalysis plot saved -> {analysis_path}")

## Cell 11: Save Final Dataset

Saves both the raw and augmented datasets as compressed `.npz` files,
plus metadata as JSON. Compatible with existing training notebooks.

In [ ]:
# --- Save compressed numpy ---
npz_path = os.path.join(DATASET_DIR, 'connect4_deep_search.npz')
np.savez_compressed(
    npz_path,
    X_train=X_aug,
    y_train=y_aug,
)
print(f"Augmented dataset saved -> {npz_path}")

# Also save original (pre-augmentation) for reference
npz_raw_path = os.path.join(DATASET_DIR, 'connect4_deep_search_raw.npz')
np.savez_compressed(
    npz_raw_path,
    X_train=X_raw,
    y_train=y_raw,
)
print(f"Raw dataset saved      -> {npz_raw_path}")

# --- Metadata ---
metadata = {
    'total_games': len(game_metas),
    'total_examples_raw': len(X_raw),
    'total_examples_augmented': len(X_aug),
    'encoding': '6x7x2 (option b)',
    'augmentation': 'horizontal_flip',
    'depth_bands': DEPTH_BANDS,
    'avg_mcts_depth': float(np.mean([m['avg_depth'] for m in game_metas])) if game_metas else 0,
    'random_prob': RANDOM_PROB,
    'random_depth': RANDOM_DEPTH,
    'strong_epsilon': STRONG_EPSILON,
    'max_runtime_minutes': MAX_RUNTIME_MINUTES,
    'num_workers': NUM_WORKERS,
    'seed': SEED,
    'generation_date': datetime.now().isoformat(),
    'game_results': {
        'p1_wins': sum(1 for m in game_metas if m['result'] == 'p1_win'),
        'p2_wins': sum(1 for m in game_metas if m['result'] == 'p2_win'),
        'draws': sum(1 for m in game_metas if m['result'] == 'draw'),
    },
}

meta_path = os.path.join(DATASET_DIR, 'deep_search_metadata.json')
with open(meta_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"Metadata saved         -> {meta_path}")

# --- File sizes ---
print(f"\nFile sizes:")
for fname in ['connect4_deep_search.npz', 'connect4_deep_search_raw.npz', 'deep_search_metadata.json']:
    fpath = os.path.join(DATASET_DIR, fname)
    if os.path.exists(fpath):
        size_mb = os.path.getsize(fpath) / (1024**2)
        print(f"  {fname}: {size_mb:.1f} MB")

## Cell 12: Merge with Existing Dataset (Optional)

If you have a previously generated dataset (e.g. from the ULTRAFAST or Production notebooks),
you can merge it with the deep search dataset here. Update `EXISTING_DATASET_PATH` below.

In [ ]:
# ========== OPTIONAL: Merge with existing dataset ==========
# Set this to the path of your existing .npz file, or None to skip

EXISTING_DATASET_PATH = None  # e.g. '/content/drive/MyDrive/Connect4_FAST/datasets/connect4_fast.npz'

if EXISTING_DATASET_PATH and os.path.exists(EXISTING_DATASET_PATH):
    print(f"Loading existing dataset from: {EXISTING_DATASET_PATH}")
    existing = np.load(EXISTING_DATASET_PATH)

    X_existing = existing['X_train']
    y_existing = existing['y_train']
    print(f"  Existing: X={X_existing.shape}, y={y_existing.shape}")

    # Concatenate
    X_merged = np.concatenate([X_existing, X_aug], axis=0)
    y_merged = np.concatenate([y_existing, y_aug], axis=0)

    # Shuffle
    indices = np.random.permutation(len(X_merged))
    X_merged = X_merged[indices]
    y_merged = y_merged[indices]

    print(f"  Deep search: X={X_aug.shape}, y={y_aug.shape}")
    print(f"  Merged:      X={X_merged.shape}, y={y_merged.shape}")

    # Save merged dataset
    merged_path = os.path.join(DATASET_DIR, 'connect4_merged.npz')
    np.savez_compressed(merged_path, X_train=X_merged, y_train=y_merged)
    print(f"\n  Merged dataset saved -> {merged_path}")
    print(f"  Size: {os.path.getsize(merged_path)/(1024**2):.1f} MB")
else:
    if EXISTING_DATASET_PATH:
        print(f"Existing dataset not found at: {EXISTING_DATASET_PATH}")
    else:
        print("No existing dataset path specified -- skipping merge.")
    print("To merge later, set EXISTING_DATASET_PATH and re-run this cell.")

## How to Load in Your Training Notebook

Copy the cell below into your training notebook to load the deep search dataset.

In [ ]:
# Example: Load the deep search dataset in your training notebook
# Adjust the path to match your setup

data = np.load(os.path.join(DATASET_DIR, 'connect4_deep_search.npz'))
X_train = data['X_train']
y_train = data['y_train']

print(f"Loaded deep search dataset:")
print(f"  X shape: {X_train.shape}")   # (N, 6, 7, 2)
print(f"  y shape: {y_train.shape}")   # (N,)
print(f"\nReady for training!")